In [2]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import base64
import pandas as pd

# Import AnimalShelter class
from crud import AnimalShelter

# Initialize AnimalShelter object
username = "root"
password = "KLChD61hul"
shelter = AnimalShelter(username, password)

# Retrieve all data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column
df.drop(columns=['_id'], inplace=True)

# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    # Unique Identifier Header
    html.H2("Dashboard by Francis Cottrell-Eshaghi"),
    
    # Module Header
    html.H1("Module 5 Assignment - Francis Cottrell-Eshaghi"),
    
    # Filter options
    html.Div([
        dcc.Dropdown(
            id='filter-outcome-type',
            options=[{'label': i, 'value': i} for i in df['outcome_type'].unique()],
            multi=False,
            placeholder="Select Outcome Type",
            value=df['outcome_type'].unique()[0]
        ),
        dcc.Dropdown(
            id='filter-breed',
            options=[{'label': i, 'value': i} for i in df['breed'].unique()],
            multi=False,
            placeholder="Select Breed",
            value=df['breed'].unique()[0]
        ),
    ]),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],
        page_size=10,
        sort_action='native',
        filter_action='native',
        editable=True,
        style_table={'overflowX': 'scroll'},
        style_cell={
            'minWidth': '0px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        },
        fixed_rows={'headers': True, 'data': 0},
        virtualization=True,
        page_action='native',
        page_current=0,
    ),
    
    # Geolocation chart and another chart
    html.Div([
        html.Div(id='map-id', className='col s12 m6'),
        html.Div(id='graph-id', className='col s12 m6'),
    ]),
])

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-outcome-type', 'value'),
     Input('filter-breed', 'value')])
def update_table(outcome_type, breed):
    if outcome_type and breed:
        data = shelter.read({'outcome_type': outcome_type, 'breed': breed})
    elif outcome_type:
        data = shelter.filter_by_outcome_type(outcome_type)
    elif breed:
        data = shelter.filter_by_breed(breed)
    else:
        data = shelter.read({})
    
    df = pd.DataFrame.from_records(data)
    df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records')

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None or index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    
    return [
        dl.Map(style={'width': '100%', 'height': '50vh'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graph(viewData):
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    
    return dcc.Graph(figure=fig)

# Run app and display result inline in the notebook
app.run_server(mode='inline', debug=True)
